In [ ]:
import sys
from pathlib import Path

root_dir = Path().absolute()
# Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
if root_dir.parts[-1:] == ('titanic',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
root_dir = str(root_dir) 

print(f"Root dir: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

In [ ]:
from PIL import Image
import datetime
import requests
import xgboost as xgb
import os
import hopsworks
import pandas as pd
import datetime
import joblib

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
mr = project.get_model_registry()
model_reg = mr.get_model("titanic", version=1)
feature_view = model_reg.get_feature_view()

# Downloads the model to a local temp directory (model_dir)
model_dir = model_reg.download()
# Load XGBoost model from local temp directory using joblib
model = joblib.load(model_dir + "/titanic_model.pkl")

In [ ]:
feature_view.init_batch_scoring(training_dataset_version=1)

batch_data = feature_view.get_batch_data()
batch_data

In [ ]:
y_pred = model.predict(batch_data)
passenger_survived = y_pred[y_pred.size-1]
passenger_survived

In [ ]:
passenger_details = batch_data.iloc[-1]
def remove_last_line_from_string(s):
    return s[:s.rfind('\n')]
passenger_details = remove_last_line_from_string(str(passenger_details))

with open(f"{root_dir}/docs/_includes/titanic.html", "w", newline="\n") as file:
    file.write(passenger_details)

In [ ]:
img_url = "https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/docs/titanic/assets/img/titanic_" + str(passenger_survived) + ".jpg"
img = Image.open(requests.get(img_url, stream=True).raw)            
output_img_path = f"{root_dir}/docs/titanic/assets/img/latest_titanic_survival.jpg"
img.save(output_img_path)

In [ ]:
from IPython.display import display
display(img)

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(0)
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
if dataset_api.exists("Resources/titanic") == False:
    dataset_api.mkdir("Resources/titanic")
dataset_api.upload(output_img_path, f"Resources/titanic/{str_today}", overwrite=True)
proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/titanic")